### **Urban Mafias**

##### Object: '평균'의 함정에서 벗어나기 - 도심 지역, 교외 지역 지구 온난화 효과 비교

In [2]:
### Calling packages


import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy import stats
import plotly.express as px

In [ ]:
### Data Auto-Downloading code
# data source: NCEI(National Centers for Environmental Information)


years = np.arange(1970, 2021)

station = 'Paris'           # station 위치가 바뀌면 그에 따라 정보 바꿔주기
station_n = '07149099999'
nation = 'France'

for year in years:
    url = 'https://www.ncei.noaa.gov/data/global-summary-of-the-day/access/' + str(year) + '/{}.csv'.format(station_n)
    r = requests.get(url, allow_redirects=True)
    open(station + str(year) + '.csv', 'wb').write(r.content)

##### Code ver.1

In [ ]:
# 다운로드한 data 중 연도별로 평균한 후 그 값들을 T_avg list에 저장


T_avg=[]

for i in years:
    file_r = open('/content/{}'.format(station)+str(i)+'.csv', 'r')
    
    log = file_r.readline().strip().split(',')
    lines = file_r.readlines()
    n_data = len(lines)
    
    TEMP=[]

    for j in range(n_data-6):
        temp = lines[j].strip().split('\"')
        TEMP.append(temp[13])
        T_a = np.array(TEMP).astype(np.float)
    
    T_avg.append((np.average(T_a)-32)*5/9) # Fahrenheit -> Celsius

print(T_avg)

In [ ]:
# 연도 list 생성


D=[]
for d in range(1950, 2021):
  D.append(d)

In [ ]:
T_data={
    "Temperature": T_avg,      # y값에 들어갈 온도(T_avg)
    "Time": D                  # x값에 들어갈 연도(D)
}

df=pd.DataFrame(data=T_data)

fit_weight=np.polyfit(df['Temperature'],df["Time"],1)
trend_f=np.poly1d(fit_weight)  # 추세선

plt.figure(figsize=(10, 6))
plt.plot(D, T_avg, 'r', label = 'Original data')
plt.plot(trend_f(df['Temperature']), df['Temperature'], 'b', label='Slope={:.6f}'.format(fit_weight[0]))
plt.title('Yearly Average Temperature Time Series of {0}, {1}'.format(station, nation))
plt.xlabel('Time')
plt.ylabel('Temperature [$^o$C]')
plt.legend(fontsize=8)
plt.show()

##### Code ver.2

In [ ]:
# 함수를 정의해서 사용하기


# missing value: 9999.9
# Temperature unit: Fahrenheit


def gen_temp_ts(station):
    year_list = np.linspace(1970, 2020, 51)
    Temp = np.zeros(51)

    for year in range(1970, 2021):
        print(year)
        

        # 결측치를 대비해 연도별 평균 온도 array인 Temp에 NaN 대입
        Avg = np.nan
        Temp[year - 1970] = Avg


        # 에러처리 파트: 해당 연도의 제대로 된 csv 파일이 존재할 경우 그 파일을 불러오고 그렇지 않을 경우 다음 연도로 넘어가기
        try:
            df = pd.read_csv(os.path.join(r'%s%d.csv' % (station, year))) 
            file = np.array(df)
        except IOError as e:
            continue


        # 연평균 온도 구하기 위해 해당 연도의 일수 체크하는 과정
        day_num = np.shape(file[:, 6])[0]
        print(day_num, end=', ')


        # 결측치로 인해 의미 있는 평균 온도를 구할 수 있는 날이 360일을 넘지 않는다면 아래 코드 실행하지 않고 건너뜀
        if day_num < 360:
            continue


        # 결측치 혹은 이상치가 있을 경우 9999.9 대입, 연평균 온도 구할 때 해당 값을 제외하고 구하기
        Sum = 0
        for row_index in range(day_num):
            if file[row_index, 6] == 9999.9:
                day_num -= 1
                continue
            Sum += (file[row_index, 6] - 32) * 5 / 9  # Fahrenheit -> Celsius
        Avg = Sum / day_num
        print(Avg)
        Temp[year - 1970] = Avg


    # 1970~2020 평균 온도를 기준값으로 하여 차이 구하기(온도 상승 폭을 고려)
    ref_temp = np.nanmean(Temp)
    print(ref_temp)
    

    # NaN을 제외하고 선형 추세선 그리기
    mask = ~np.isnan(year_list) & ~np.isnan(Temp)
    # stats.linregress: 기울기, y절편 및 기타 통계적 상수 구할 수 있는 함수
    slope, intercept, r_value, p_value, std_err = stats.linregress(year_list[mask], Temp[mask])


    # Make plot
    plt.figure(figsize=(10,6))
    plt.plot(year_list, Temp - ref_temp, 'r', label='Original data')
    regression_line = slope * year_list + intercept - ref_temp
    plt.plot(year_list, regression_line, 'b', label='Slope = {:.6f}'.format(slope))
    plt.title('Yearly Average Temperature Time Series of %s, Japan' % station)
    plt.xlabel('Time')
    plt.ylabel('Temperature [$^o$C]')
    plt.ylim(ymax=2, ymin=-2)
    plt.legend(fontsize=8)
    plt.savefig(r'D:\%s_warming_time_series.png' % station)
    plt.show()

In [ ]:
# 온도 추세를 보고자 하는 관측소의 정보가 담긴 dictionary 생성


station_dict = {'Mishima': 47657, 'Tokyo': 47662, 'Mito': 47629, 'Chiba': 47682, 'Omaezaki': 47655, 'Shizuoka': 47656}
# station_dict = {'Sidney': 94768, 'Nowra': 94750, 'Woomera': 94659, 'williamtown': 94776}
# station_dict = {'Moscow': 27612, 'Elatma': 27648, 'Pavelec': 27823, 'Vologda': 27037, 'Kazan': 27595}


# 앞서 만든 함수 적용

for station in station_dict.keys():
    print(station)
    gen_temp_ts(station)

In [ ]:
### Clearing files code


for station in station_dict.keys():
    for year in range(1970, 2021):
        os.remove('%s%d.csv' % (station, year))